In [ ]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
pkg"precompile"

In [ ]:
using MotionCaptureJointCalibration
using RigidBodyDynamics
using ValkyrieRobot
using Ipopt
using Random

Create a `Mechanism` (we'll use the NASA Valkyrie robot) and `MechanismState`:

In [ ]:
val = Valkyrie()
mechanism = val.mechanism
remove_fixed_tree_joints!(mechanism)
state = MechanismState(mechanism)

Let's generate a random calibration problem, as well as its ground truth solution:

In [ ]:
using MotionCaptureJointCalibration.SyntheticDataGeneration
bodies_with_markers = findbody.(Ref(mechanism), ["leftFoot", "pelvis"])
Random.seed!(1)
body_weights = Dict(b => rand() for b in bodies_with_markers)
problem, groundtruth = generate_calibration_problem(state, body_weights);

In [ ]:
problem

In [ ]:
groundtruth

Solve the calibration problem (without using the ground truth result):

In [ ]:
solver = IpoptSolver(print_level = 0, max_iter = 10000, check_derivatives_for_naninf = "yes", tol = 1e-10)
result = solve(problem, solver)

And visualize the results:

In [ ]:
using RigidBodyTreeInspector
using Interact
using DrakeVisualizer

In [ ]:
#NBSKIP
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));

In [ ]:
vis = Visualizer()[:valkyrie]
geometry = visual_elements(mechanism, URDFVisuals(ValkyrieRobot.urdfpath(); package_path = [ValkyrieRobot.packagepath()]))
setgeometry!(vis, mechanism, geometry)
state = MechanismState(mechanism)

In [ ]:
inspect!(state, vis, problem, result)